# Chapter 3 — Parallel Programming with Processes (Extracted Code)

Questa raccolta contiene esempi eseguibili estratti dal capitolo 3. Alcuni esempi MPI richiedono **mpi4py** e l'esecuzione con `mpiexec`.

## Introduzione ai processi e multiprocessing

In [ ]:
import multiprocessing
import time

def function(i):
  print ("start Process %i" %i)
  time.sleep(2)
  print ("end Process %i" %i)
  return

if __name__ == '__main__':
  p1 = multiprocessing.Process(target=function, args=(1,))
  p2 = multiprocessing.Process(target=function, args=(2,))
  p3 = multiprocessing.Process(target=function, args=(3,))
  p4 = multiprocessing.Process(target=function, args=(4,))
  p5 = multiprocessing.Process(target=function, args=(5,))
  p1.start(); p2.start(); p3.start(); p4.start(); p5.start()
  p1.join(); p2.join(); p3.join(); p4.join(); p5.join()
  print("END Program")

In [ ]:
import multiprocessing
import time

def function(i):
  print ("start Process %i" %i)
  time.sleep(2)
  print ("end Process %i" %i)
  return

if __name__ == '__main__':
  processes = []
  n_procs = 5
  for i in range(n_procs):
    p = multiprocessing.Process(target=function, args=(i,))
    processes.append(p)
    p.start()
  for i in range(n_procs):
    processes[i].join()
  print("END Program")

In [ ]:
import multiprocessing
import os
import time

def function():
  pid = os.getpid()
  print ("start Process %s" %pid)
  time.sleep(2)
  print ("end Process %s" %pid)
  return

if __name__ == '__main__':
  processes = []
  n_procs = 5
  for i in range(n_procs):
    p = multiprocessing.Process(target=function)
    processes.append(p)
    p.start()
  for i in range(n_procs):
    processes[i].join()
  print("END Program")

In [ ]:
from multiprocessing import Process
import time

class ChildProcess(Process):
  def __init__(self, count):
    Process.__init__(self)
    self.count = count
  
  def run(self):
    print ("start Process %s" %self.count)
    time.sleep(2)
    print ("end Process %s" %self.count)

if __name__ == '__main__':
  processes = []
  n_procs = 5
  for i in range(n_procs):
    p = ChildProcess(i)
    processes.append(p)
    p.start()
  for i in range(n_procs):
    processes[i].join()

## Pool di processi (multiprocessing.Pool)

In [ ]:
import multiprocessing
import time

def function(i):
  process = multiprocessing.current_process()
  print ("start Process %i(pid:%s)" %(i,process.pid))
  time.sleep(2)
  print ("end Process %i(pid:%s)" %(i,process.pid))
  return

if __name__ == '__main__':
  pool = multiprocessing.Pool()
  print("Processes started: %s" %pool._processes)
  for i in range(pool._processes):
    results = pool.apply(function, args=(i,))
  pool.close()
  print("END Program")

In [ ]:
import multiprocessing
import time

def function(i):
  process = multiprocessing.current_process()
  print ("start Task %i(pid:%s)" %(i,process.pid))
  time.sleep(2)
  print ("end Task %i(pid:%s)" %(i,process.pid))
  return

if __name__ == '__main__':
  pool = multiprocessing.Pool(processes=4)
  print("Processes started: %s" %pool._processes)
  for i in range(12):
    results = pool.apply(function, args=(i,))
  pool.close()
  print("END Program")

In [ ]:
from multiprocessing.pool import Pool
import time, math

def func(value):
    result = math.sqrt(value)
    print(f"The value {value} and the elaboration is {result}")
    time.sleep(0.2)  # shorter sleep to keep demo snappy
    return result

if __name__ == '__main__':
    data = [10, 3, 6, 1]
    with Pool() as pool:
        results = pool.map(func, data)
        print("The main process is going on...")
        for r in results:
            print(f"This is the result: {r}")
    print("END Program")


In [ ]:
from multiprocessing.pool import Pool
import time, math

def func(value):
    result = math.sqrt(value)
    print(f"The value {value} and the elaboration is {result}")
    time.sleep(0.2)
    return result

if __name__ == '__main__':
    data = [10, 3, 6, 1]
    with Pool() as pool:
        results = pool.map_async(func, data)
        print("Main Process is going on...")
        for r in results.get():
            print(f"This is the result: {r}")
    print("END Program")


In [ ]:
from multiprocessing.pool import Pool
import time, math

def func(value):
    result = math.sqrt(value)
    print(f"The value {value} and the elaboration is {result}")
    time.sleep(0.1)
    return result

if __name__ == '__main__':
    data = [10,3,6,1,4,5,2,9,7,3,4,6]
    with Pool() as pool:
        results = pool.map(func, data, chunksize=4)
        print("The main process is going on...")
        for r in results:
            print(f"This is the result: {r}")
    print("END Program")


## ProcessPoolExecutor (concurrent.futures)

In [ ]:
import time, math
from concurrent.futures import ProcessPoolExecutor

def func(value):
    result = math.sqrt(value)
    print(f"The value {value} and the elaboration is {result}")
    time.sleep(0.2)
    return result

if __name__ == '__main__':
    with ProcessPoolExecutor(4) as executor:
        data = [10,3,6,1]
        for result in executor.map(func, data):
            print(f"This is the result: {result}")
    print("END Program")


In [ ]:
import time, math, os
from concurrent.futures import ProcessPoolExecutor

def func(value):
    result = math.sqrt(value)
    pid = os.getpid()
    print(f"[Pid:{pid}] The value {value} and the elaboration is {result}")
    time.sleep(0.1)
    return result

if __name__ == '__main__':
    with ProcessPoolExecutor(4) as executor:
        data = [10,3,6,1,4,5,2,9,7,3,4,6]
        for r in executor.map(func, data, chunksize=4):
            print(f"This is the result: {r}")
    print("END Program")


## Canali di comunicazione tra processi

In [ ]:
from multiprocessing import Process, Queue
import time, random

class Consumer(Process):
  def __init__(self, count, queue):
    Process.__init__(self)
    self.count = count
    self.queue = queue
  
  def run(self):
    for i in range(self.count):
      local = self.queue.get()
      time.sleep(2)
      print("consumer has used this: %s" %local)

class Producer(Process):
  def __init__(self, count, queue):
    Process.__init__(self)
    self.count = count
    self.queue = queue

  def request(self):
    time.sleep(1)
    return random.randint(0,100)
 
  def run(self):
    for i in range(self.count):
      local = self.request()
      self.queue.put(local)
      print("producer has loaded this: %s" %local)

if __name__ == '__main__':
  queue = Queue()
  count = 5
  p1 = Producer(count, queue)
  p2 = Consumer(count, queue)
  p1.start(); p2.start()
  p1.join(); p2.join()

In [ ]:
from multiprocessing import Process, Pipe
import time, random

class Consumer(Process):
  def __init__(self, count, conn):
    Process.__init__(self)
    self.count = count
    self.conn = conn
  
  def run(self):
    for i in range(self.count):
      local = self.conn.recv()
      time.sleep(2)
      print("consumer has used this: %s" %local)

class Producer(Process):
  def __init__(self, count, conn):
    Process.__init__(self)
    self.count = count
    self.conn = conn

  def request(self):
    time.sleep(1)
    return random.randint(0,100)
 
  def run(self):
    for i in range(self.count):
      local = self.request()
      self.conn.send(local)
      print("producer has loaded this: %s" %local)

if __name__ == '__main__':
  recver, sender = Pipe()
  count = 5
  p1 = Producer(count, sender)
  p2 = Consumer(count, recver)
  p1.start(); p2.start()
  p1.join(); p2.join()
  recver.close(); sender.close()

## mpi4py — esempi base e comunicazioni

In [ ]:
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.rank
print("The process %d is started" % rank)


In [ ]:
import time
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.rank
print("The process %d is started" % rank)
time.sleep(10)
print("The process %d is ended" % rank)


In [ ]:
import time
for i in range(4):
    print("The process %d is started" % i)
    time.sleep(10)
    print("The process %d is ended" % i)


In [ ]:
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.rank
print("The process %d is started" % rank)
n = 0
for i in range(1000000):
    n += i
print("The process %d is ended" % rank)


In [ ]:
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.rank
print("Process %s started" % rank)
if rank == 0:
    msg = "This is the message"
    receiver = 1
    comm.send(msg, dest=receiver)
    print("Process 0 sent a message to %d: %s" % (receiver, msg))
elif rank == 1:
    source = 0
    msg = comm.recv(source=source)
    print("Process 1 received a message from %d: %s" % (source, msg))


In [ ]:
from mpi4py import MPI
import random
comm = MPI.COMM_WORLD
rank = comm.rank
if rank == 0:
    data = random.randint(1, 10)
else:
    data = None
data = comm.bcast(data, root=0)
if rank == 1:
    print(f"The square of {data} is {data*data}")
elif rank == 2:
    print(f"Half of {data} is {data/2}")
elif rank == 3:
    print(f"Double of {data} is {data*2}")


In [ ]:
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.rank
if rank == 0:
   array = ['AAA','BBB','CCC','DDD']
else:
   array = None
data = comm.scatter(array, root=0)
print("Process %d is working on %s element" % (rank, data))


In [ ]:
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.rank
if rank == 0:
    data = 'AAA'
elif rank == 1:
    data = 'BBB'
elif rank == 2:
    data = 'CCC'
elif rank == 3:
    data = 'DDD'
array = comm.gather(data, root=0)
if rank == 0:
    print("The new array is %s " % array)


In [ ]:
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.rank
size = comm.size 
data = 2*rank + 1
print("Process %d calculated this value: %d" % (rank, data))
array = comm.gather(data, root=0)
if rank == 0:
   result = sum(array)
   print("The result of the parallel computation is %d" % result)


In [ ]:
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.rank
if rank == 0:
    output = ['0A','0B','0C','0D']
elif rank == 1:
    output = ['1A','1B','1C','1D']
elif rank == 2:
    output = ['2A','2B','2C','2D']
elif rank == 3:
    output = ['3A','3B','3C','3D']
input = comm.alltoall(output)
print("Process %s received %s" % (rank, input))


In [ ]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.rank

if rank == 0:
    output = np.array([0,0,0,0])
elif rank == 1:
    output = np.array([1,1,1,1])
elif rank == 2:
    output = np.array([2,2,2,2])
elif rank == 3:
    output = np.array([3,3,3,3])
print("Process %d. Sending %s" % (rank, output))
input = comm.reduce(output, root=0, op=MPI.SUM)
if rank == 0:
    print("The result of the parallel computation is %s" % input)


In [ ]:
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.rank

comm_3D = comm.Create_cart(dims=[3,3,3], periods=[False,False,False], reorder=False)
xyz = comm_3D.Get_coords(rank)
print("In this 3D topology, process %s has coordinates %s " % (rank, xyz))


In [ ]:
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.rank

comm_3D = comm.Create_cart(dims=[3,3,3], periods=[False,False,False], reorder=False)
xyz = comm_3D.Get_coords(rank)

if rank == 12:
  print("In this 3D topology, process %s has coordinates %s " % (rank, xyz))
  right,left = comm_3D.Shift(0,1)
  up,down = comm_3D.Shift(1,1)
  forward,backward = comm_3D.Shift(2,1)
  print("Neighbors (left-right): %s %s" % (left, right))
  print("Neighbors (up-down): %s %s" % (up, down))
  print("Neighbors (forward-backward): %s %s" % (forward, backward))


## Threading vs Multiprocessing (confronti)

In [ ]:
import threading, multiprocessing, time

def io_bound_task(seconds):
    print(f"{threading.current_thread().name} is working...")
    time.sleep(seconds)
    print(f"{threading.current_thread().name} ended its work.")

def threading_example():
    start_time = time.time()
    threads = []
    for _ in range(5):
        t = threading.Thread(target=io_bound_task, args=(1,))
        threads.append(t); t.start()
    for t in threads: t.join()
    print(f"Time with threading: {time.time() - start_time:.2f} seconds")

def multiprocessing_example():
    start_time = time.time()
    procs = []
    for _ in range(5):
        p = multiprocessing.Process(target=time.sleep, args=(1,))
        procs.append(p); p.start()
    for p in procs: p.join()
    print(f"Time with multiprocessing: {time.time() - start_time:.2f} seconds")

if __name__ == '__main__':
    print("Execution of I/O-bound task:")
    print("Using threading..."); threading_example()
    print("Using multiprocessing..."); multiprocessing_example()


In [ ]:
import threading, multiprocessing, time, math

def count_primes(n):
    count = 0
    for num in range(2, n):
        is_prime = True
        r = int(math.sqrt(num)) + 1
        for j in range(2, r):
            if num % j == 0:
                is_prime = False
                break
        if is_prime:
            count += 1
    print(f"Prime numbers found: {count}")

def threading_example_cpu():
    start = time.time()
    threads = [threading.Thread(target=count_primes, args=(20000,)) for _ in range(4)]
    [t.start() for t in threads]
    [t.join() for t in threads]
    print(f"Time with threading: {time.time()-start:.2f} seconds")

def multiprocessing_example_cpu():
    start = time.time()
    procs = [multiprocessing.Process(target=count_primes, args=(20000,)) for _ in range(4)]
    [p.start() for p in procs]
    [p.join() for p in procs]
    print(f"Time with multiprocessing: {time.time()-start:.2f} seconds")

if __name__ == '__main__':
    print("Execution of CPU-bound task:")
    print("Using threading..."); threading_example_cpu()
    print("Using multiprocessing..."); multiprocessing_example_cpu()


## Ottimizzazioni avanzate multiprocessing

In [ ]:
# Linux-only demo: requires permissions and may not work on all environments.
import os, multiprocessing

def cpu_bound_task(task_id):
    print(f"Process {task_id} assigned to CPU {os.sched_getaffinity(0)}")
    result = sum(i*i for i in range(10**6))
    print(f"Process {task_id} finished, result: {result}")

if __name__ == '__main__':
    processes = []
    for i in range(4):
        p = multiprocessing.Process(target=cpu_bound_task, args=(i,))
        p.start()
        cpu_core = i % os.cpu_count()
        try:
            os.sched_setaffinity(p.pid, {cpu_core})
        except AttributeError:
            print("sched_setaffinity not available on this platform.")
        processes.append(p)
    for p in processes:
        p.join()
    print("All processes are completed.")


In [ ]:
# Linux-only; may require elevated privileges to set high priority.
import os, multiprocessing

def intensive_task():
    print(f"Process ID: {os.getpid()} with Priority: {os.getpriority(os.PRIO_PROCESS, 0)}")
    result = sum(i*i for i in range(10**6))
    print(f"Result: {result}")

if __name__ == "__main__":
    p = multiprocessing.Process(target=intensive_task)
    p.start()
    try:
        os.setpriority(os.PRIO_PROCESS, p.pid, -10)  # -20..19
    except PermissionError:
        print("Permission denied: cannot change priority without proper privileges.")
    p.join()
    print("Process completed.")


In [ ]:
import multiprocessing

def increment(shared_counter, lock):
    with lock:
        shared_counter.value += 1
        print(f"Counter incremented to {shared_counter.value}")

if __name__ == "__main__":
    counter = multiprocessing.Value('i', 0)
    lock = multiprocessing.Lock()
    procs = [multiprocessing.Process(target=increment, args=(counter, lock)) for _ in range(10)]
    [p.start() for p in procs]
    [p.join() for p in procs]
    print(f"Final counter value: {counter.value}")


In [ ]:
import multiprocessing, time

def process_task(task_id):
    print(f"Starting the task {task_id} in the process {multiprocessing.current_process().name}")
    time.sleep(0.5)
    print(f"Task {task_id} completed")
    return f"Task result {task_id}"

if __name__ == "__main__":
    with multiprocessing.Pool(processes=4) as pool:
        tasks = [pool.apply_async(process_task, args=(i,)) for i in range(10)]
        results = [t.get() for t in tasks]
    print("Results:", results)


In [ ]:
import multiprocessing, time

def square(x):
    time.sleep(0.2)
    return x*x

if __name__ == "__main__":
    numbers = [1,2,3,4,5,6,7,8]
    with multiprocessing.Pool(processes=4) as pool:
        results_sync = pool.map(square, numbers)
    print("Synchronous results:", results_sync)

    with multiprocessing.Pool(processes=4) as pool:
        result_async = pool.map_async(square, numbers)
        results_async = result_async.get()
    print("Asynchronous results:", results_async)


In [ ]:
import multiprocessing, time, queue

def dynamic_task(q):
    while True:
        try:
            task_id = q.get_nowait()
        except queue.Empty:
            break
        print(f"Task execution {task_id} in the process {multiprocessing.current_process().name}")
        time.sleep(0.5)
        print(f"Task {task_id} completed")
        q.task_done()

if __name__ == "__main__":
    task_queue = multiprocessing.JoinableQueue()
    for i in range(10):
        task_queue.put(i)
    processes = [multiprocessing.Process(target=dynamic_task, args=(task_queue,)) for _ in range(4)]
    [p.start() for p in processes]
    task_queue.join()
    [p.join() for p in processes]
    print("All tasks have been completed.")


In [ ]:
# Process-safe priority scheduling using a JoinableQueue of (priority, task) tuples.
import multiprocessing, time, queue

def worker(pq):
    while True:
        try:
            priority, task_id = pq.get_nowait()
        except queue.Empty:
            break
        print(f"Task execution {task_id} (Priority {priority}) in {multiprocessing.current_process().name}")
        time.sleep(0.3)
        print(f"Task {task_id} completed")
        pq.task_done()

if __name__ == "__main__":
    pq = multiprocessing.JoinableQueue()
    # lower number == higher priority
    tasks = [(3, "Low"), (2, "High"), (4, "Very Low"), (1, "Very High")]
    for item in sorted(tasks):  # pre-sort before enqueue to maintain priority order
        pq.put(item)

    processes = [multiprocessing.Process(target=worker, args=(pq,)) for _ in range(2)]
    [p.start() for p in processes]
    pq.join()
    [p.join() for p in processes]
    print("All priority tasks have been completed.")
